In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 46.9 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 8.6 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595740 sha256=6bc71cbcd7884a3528a15cb82a450deb410fd4e3f12fc6960fc1a92c1d498904
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 9.3 MB/s 
     |████████████████████████████████| 754 kB 7.3 MB/s 
     |████████████████████████████████| 3.0 MB 75.1 MB/s 
     |████████████████████████████████| 895 kB 66.1 MB/s 


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-d56h8vvn
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-d56h8vvn
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=10e49dd499890632d82fe219aed1c95f1d546f8ad464c6193379013c43a77dfb
  Stored in directory: /tmp/pip-ephem-wheel-cache-ao1at103/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/NLP Assignment2/ratings_train.txt", sep='\t')
test = pd.read_csv("/content/drive/MyDrive/NLP Assignment2/ratings_test.txt", sep='\t')

train = train.dropna()
test = test.dropna()

print(train.shape)
print(test.shape)

(149995, 3)
(49997, 3)


In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
test.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [ ]:
train_data_list = []
for document, label in zip(train['document'], train['label'])  :
    data = []
    data.append(document)
    data.append(str(label))

    train_data_list.append(data)
train_data_list[:5]

[['아 더빙.. 진짜 짜증나네요 목소리', '0'],
 ['흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'],
 ['너무재밓었다그래서보는것을추천한다', '0'],
 ['교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'],
 ['사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '1']]

In [ ]:
test_data_list = []
for document, label in zip(test['document'], test['label'])  :
    data = []
    data.append(document)
    data.append(str(label))

    test_data_list.append(data)
test_data_list[:5]

[['굳 ㅋ', '1'],
 ['GDNTOPCLASSINTHECLUB', '0'],
 ['뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아', '0'],
 ['지루하지는 않은데 완전 막장임... 돈주고 보기에는....', '0'],
 ['3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??', '0']]

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 142
batch_size = 32
warmup_ratio = 0.1
num_epochs = 4
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_data_list, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_data_list, 0, 1, tok, max_len, True, False)

using cached model


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids) # 실제 단어 토큰이 있는 위치의 길이 만큼만 1로 설정
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

모델

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
#optimizer 설정
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
#schedule 설정
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
#정확도 측정
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

KoBERT모델 학습

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/4688 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.7481940984725952 train acc 0.5625
epoch 1 batch id 201 loss 0.6852412819862366 train acc 0.5205223880597015
epoch 1 batch id 401 loss 0.47014105319976807 train acc 0.6183759351620948
epoch 1 batch id 601 loss 0.34428223967552185 train acc 0.6790765391014975
epoch 1 batch id 801 loss 0.4414731562137604 train acc 0.7156679151061174
epoch 1 batch id 1001 loss 0.33559730648994446 train acc 0.742288961038961
epoch 1 batch id 1201 loss 0.5148530006408691 train acc 0.759757493755204
epoch 1 batch id 1401 loss 0.2039947807788849 train acc 0.7730415774446824
epoch 1 batch id 1601 loss 0.6218390464782715 train acc 0.7824601811367895
epoch 1 batch id 1801 loss 0.6134650707244873 train acc 0.7900645474736258
epoch 1 batch id 2001 loss 0.26667457818984985 train acc 0.7955084957521239
epoch 1 batch id 2201 loss 0.553216278553009 train acc 0.8010279418446161
epoch 1 batch id 2401 loss 0.18142463266849518 train acc 0.8057189712619742
epoch 1 batch id 2601 loss 0.2901523411273

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 3 test acc 0.8970037280377972


  0%|          | 0/4688 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.40806999802589417 train acc 0.90625
epoch 4 batch id 201 loss 0.03505726903676987 train acc 0.9583333333333334
epoch 4 batch id 401 loss 0.0524241179227829 train acc 0.9589307980049875
epoch 4 batch id 601 loss 0.08702141791582108 train acc 0.9597545757071547
epoch 4 batch id 801 loss 0.1126987487077713 train acc 0.9611813358302123
epoch 4 batch id 1001 loss 0.044576793909072876 train acc 0.9618818681318682
epoch 4 batch id 1201 loss 0.46564093232154846 train acc 0.9626092839300583
epoch 4 batch id 1401 loss 0.09857282042503357 train acc 0.9627721270521057
epoch 4 batch id 1601 loss 0.33706700801849365 train acc 0.9628162086196127
epoch 4 batch id 1801 loss 0.10986854135990143 train acc 0.9636139644641866
epoch 4 batch id 2001 loss 0.00952197052538395 train acc 0.9644708895552224
epoch 4 batch id 2201 loss 0.4380466043949127 train acc 0.9649449114039074
epoch 4 batch id 2401 loss 0.02294306829571724 train acc 0.9655091628488129
epoch 4 batch id 2601 loss 0.034

  0%|          | 0/1563 [00:00<?, ?it/s]

epoch 4 test acc 0.8983632929770166


새로운 리뷰 테스트

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("부정적")

            elif np.argmax(logits) == 1:
                test_eval.append("긍정적")

        print("이 영화 리뷰는 " + test_eval[0]+ "인 리뷰입니다.")

In [ ]:
while True:
    review = input("리뷰 입력 : ")
    if review == "exit" :
        break
    else:
        predict(review)

    print("\n")

리뷰 입력 : 간만에 감동 준 영화ㅠㅠ
이 영화 리뷰는 긍정적인 리뷰입니다.


리뷰 입력 : 잔잔하고 마음 따뜻해지는 영화입니다. 배우들의 연기가 좋았어요.
이 영화 리뷰는 긍정적인 리뷰입니다.


리뷰 입력 : 훈훈한 영화에요 가족들이랑 같이 보러가서 여러번 눈물 흘렸습니다
이 영화 리뷰는 긍정적인 리뷰입니다.


리뷰 입력 : 잔잔하고 마음 따뜻해지는 영화입니다. 배우들의 연기가 좋았어요.
이 영화 리뷰는 긍정적인 리뷰입니다.


리뷰 입력 : 감동이에요.. 마음이 따뜻해지는 영화
이 영화 리뷰는 긍정적인 리뷰입니다.


리뷰 입력 : 배우님들 연기도 정말 다 최고구 영상미연출스토리 머하나 빠지는 게 없는 영화입니다. 형제자매가 있다면 더 공감갈만한 영화였어요 저는 정말 재미있게 봤네요
이 영화 리뷰는 긍정적인 리뷰입니다.


리뷰 입력 : 아주 눈물콧물 다뺌 휴지 안챙겨 가서 낭패봤어요
이 영화 리뷰는 긍정적인 리뷰입니다.


리뷰 입력 : 진짜 영화로 만들어줘서 너무 감동입니다!보는내내 시간 안본거 처음입니다이 영화보구 아무도 안당했으면 좋겠습니다!변요한 김무열 최고네요 !
이 영화 리뷰는 긍정적인 리뷰입니다.


리뷰 입력 : 재밌어요. 전개가 나름빠르고 연기도 좋습니다.
이 영화 리뷰는 긍정적인 리뷰입니다.


리뷰 입력 : 지루하지않고,사실적표현등 좋았음.경각심도 불러올수있었고 good~
이 영화 리뷰는 긍정적인 리뷰입니다.


리뷰 입력 : 출연한 모든 배우들의 흑역사
이 영화 리뷰는 부정적인 리뷰입니다.


리뷰 입력 : 100억 갖다 버리는게 더 재밌겠다
이 영화 리뷰는 부정적인 리뷰입니다.


리뷰 입력 : 돈주고 봐야되는 영화가 아니라 돈받고 봐야되는 희대의 괴작
이 영화 리뷰는 부정적인 리뷰입니다.


리뷰 입력 : 와 어쩜 이 명배우들을 캐스팅해놓고 이런 똥을 쌀수있을까?이것도 능력이라면 능력이다;;
이 영화 리뷰는 부정적인 리뷰입니다.


리뷰 입력 : 학교에서 이걸 왜 보여줫죠? 덕분에 수업시간이 줄어들고 잠도 편하게잠 ^^
이 